In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/RecFldGrn


# Read Raw Data

In [2]:
import os 
import pandas as pd

RecName = 'P'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)


data/Simulations/P.csv


In [3]:

# load all records
df = pd.read_csv(raw_file_path)
df

,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


# Preprocess

TODO

# Save To Disk by Bucket

In [4]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder

In [5]:
rec_folder = 'data/ProcData/RecFolder'
fullrec_folder = os.path.join(rec_folder, RecName)
print(df.shape, fullrec_folder)

if not os.path.exists(fullrec_folder): os.makedirs(fullrec_folder)
write_df_to_folders(RecName, fullrec_folder, df)


(8, 3) data/ProcData/RecFolder/P


In [6]:
from recfldgrn.datapoint import PatientDP
print(rec_folder)

PID = 'P4'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
print(Pat)
dfx = Pat.get_df_rec('P')
dfx

data/ProcData/RecFolder


,PID,age,basicInfo
0,P4,23,Male&A


In [7]:
from recfldgrn.datapoint import load_df_data_from_folder
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder)
df

data/ProcData/RecFolder/P


,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


# Field & Grain

In [8]:
FieldGrainInfo = []

In [9]:
RecName = 'P'
prefix_ids = []
focal_ids = ['PID']
field_list = ['basicInfo', 'age']

## Field `basicInfo`

In [10]:
# (a) field and (b) get field-focused df
field = 'basicInfo'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,basicInfo
0,P0,Male&B
1,P1,Male&B
2,P2,Female&B
3,P3,Female&B
4,P4,Male&A
5,P5,Female&D
6,P6,Female&A
7,P7,Male&C


### Grain `basicInfoDftGrn`

In [11]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info

In [12]:
# (c) grain
grain = 'basicInfoDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

basicInfoDftGrn
basicInfo-basicInfoDftGrn
P@basicInfo-basicInfoDftGrn


In [13]:
grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
s = dfx[field].apply(grain_str_fn)
s

0      [Male, B]
1      [Male, B]
2    [Female, B]
3    [Female, B]
4      [Male, A]
5    [Female, D]
6    [Female, A]
7      [Male, C]
Name: basicInfo, dtype: object

In [14]:
from recfldgrn.utils import generate_grain_vocab_info
Vocab = generate_grain_vocab_info(s) # load from disk
v2idx = Vocab['v2idx']

In [15]:
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
s = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
s

0    [1, 2]
1    [1, 2]
2    [3, 2]
3    [3, 2]
4    [1, 4]
5    [3, 5]
6    [3, 4]
7    [1, 6]
Name: basicInfo, dtype: object

In [16]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


{'_padding': 0, 'Male': 1, 'B': 2, 'Female': 3, 'A': 4, 'D': 5, 'C': 6}


0    [1, 2]
1    [1, 2]
2    [3, 2]
3    [3, 2]
4    [1, 4]
5    [3, 5]
6    [3, 4]
7    [1, 6]
Name: basicInfo, dtype: object

In [17]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list

d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

## Field `Age`

In [18]:
# (a) field and (b) get field-focused df
field = 'age'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,age
0,P0,52
1,P1,21
2,P2,23
3,P3,77
4,P4,23
5,P5,73
6,P6,34
7,P7,76


### AgeNumeGrn

In [19]:
# (c) grain
grain = 'AgeNumeDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

AgeNumeDftGrn
age-AgeNumeDftGrn
P@age-AgeNumeDftGrn


In [20]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


0    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
1    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
2    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
3    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
4    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
5    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
6    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
7    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
Name: age, dtype: object

In [21]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list


d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

# Save Field Grain Info



In [22]:
df_FieldGrainInfo = pd.DataFrame(FieldGrainInfo)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,P,[],[PID],"[basicInfo, age]",basicInfo,basicInfoDftGrn,basicInfo-basicInfoDftGrn,P@basicInfo-basicInfoDftGrn,"{'idx2v': {0: '_padding', 1: 'Male', 2: 'B', 3..."
1,P,[],[PID],"[basicInfo, age]",age,AgeNumeDftGrn,age-AgeNumeDftGrn,P@age-AgeNumeDftGrn,"{'v2idx': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."


In [23]:
fldgrn_folder = 'data/ProcData/FldGrnInfo'

if not os.path.exists(fldgrn_folder): os.makedirs(fldgrn_folder)

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')
df_FieldGrainInfo.to_pickle(fullfldgrn_file)